In [2]:
# load json file with tags, patterns & responses

intents = json.loads(open("intents.json").read())

In [3]:
# create lists to store data
words = []
labels = []
docs = []
ignore_letters = ["?", "!", ".", ","]

# separate words from patterns and add to words list
for intent in intents['intents']:
	for pattern in intent['patterns']:
		word_list = nltk.word_tokenize(pattern)
		words.extend(word_list) 
		
		# associate patterns with respective tags
		docs.append(((word_list), intent['tag']))

		# append tags to labels list
		if intent['tag'] not in labels:
			labels.append(intent['tag'])

In [4]:
# store root words (lemma)
lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word)
		for word in words if word not in ignore_letters]
words = sorted(set(words))

In [5]:
# save words and labels lists to binary files

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(labels, open('labels.pkl', 'wb'))

In [6]:

training = []
output_empty = np.zeros(len(labels))

for doc in docs:
    bag = np.zeros(len(words))
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        if word in word_patterns:
            bag[words.index(word)] = 1

    output_row = np.copy(output_empty)
    output_row[labels.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffle training data
random.shuffle(training)

# Separate bag and output_row into individual arrays
bags = np.array([entry[0] for entry in training])
output_rows = np.array([entry[1] for entry in training])

# Combine bags and output_rows into a single NumPy array
training = np.column_stack((bags, output_rows))

In [7]:
training = np.array(training, dtype=object)

# split data
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [8]:
train_x = np.array([x for x, _ in training], dtype=int)
train_y = np.array([y for _, y in training], dtype=int)

# Input shape will be the shape of a single data point in train_x
input_shape = (train_x.shape[1],)  # Number of features in each data point

# create Sequential Neural Network model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

ValueError: too many values to unpack (expected 2)

In [ ]:
# Define the learning rate schedule
initial_lr = 0.01
lr_schedule = tensorflow.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_lr,
    decay_steps=1000,           # adjust this based on your preference
    decay_rate=0.96,            # adjust this based on your preference
    staircase=True
)

# Compile model with the learning rate scheduler
sgd = tensorflow.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y),
                 epochs=200, batch_size=5, verbose=1)

print("Training complete")

In [ ]:
# saving the model
model.save("chatbotmodel.h5", hist)